In [1]:
# !pip install pyspark

In [126]:
import pyspark
from pyspark.ml.feature import Imputer
import pyspark.sql.functions as F
from pyspark.sql.functions import col, sum, mean, median, mode, concat_ws, col, split
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [3]:
import numpy as np
import pandas as pd

In [40]:
pd.read_csv("cal_housing.csv")

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NaN
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NaN
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NaN
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NaN
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NaN
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,NaN
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,NaN
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,NaN
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,NaN


In [41]:
from pyspark.sql import SparkSession

In [42]:
spark = SparkSession.builder.appName("Practice").getOrCreate()

In [43]:
spark

In [44]:
df_pyspark = spark.read.csv("cal_housing.csv", header=True, inferSchema=True)

In [45]:
df_pyspark.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|           null|
|  -122.25|   37.85|              

In [46]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [11]:
df_pyspark.head(5)

[Row(customerID='7590-VHVEG', gender='Female', SeniorCitizen=0, Partner='Yes', Dependents='No', tenure=1, PhoneService='No', MultipleLines='No phone service', InternetService='DSL', OnlineSecurity='No', OnlineBackup='Yes', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Electronic check', MonthlyCharges=29.85, TotalCharges='29.85', Churn='No'),
 Row(customerID='5575-GNVDE', gender='Male', SeniorCitizen=0, Partner='No', Dependents='No', tenure=34, PhoneService='Yes', MultipleLines='No', InternetService='DSL', OnlineSecurity='Yes', OnlineBackup='No', DeviceProtection='Yes', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='One year', PaperlessBilling='No', PaymentMethod='Mailed check', MonthlyCharges=56.95, TotalCharges='1889.5', Churn='No'),
 Row(customerID='3668-QPYBK', gender='Male', SeniorCitizen=0, Partner='No', Dependents='No', tenure=2, PhoneService='Yes', MultipleL

In [47]:
df_pyspark.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [13]:
df_pyspark.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [49]:
df_pyspark.dtypes

[('longitude', 'double'),
 ('latitude', 'double'),
 ('housing_median_age', 'double'),
 ('total_rooms', 'double'),
 ('total_bedrooms', 'double'),
 ('population', 'double'),
 ('households', 'double'),
 ('median_income', 'double'),
 ('median_house_value', 'double'),
 ('ocean_proximity', 'string')]

In [50]:
df_pyspark.describe()

DataFrame[summary: string, longitude: string, latitude: string, housing_median_age: string, total_rooms: string, total_bedrooms: string, population: string, households: string, median_income: string, median_house_value: string, ocean_proximity: string]

In [51]:
df_pyspark.describe().show()

+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+---------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|   total_bedrooms|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+---------------+
|  count|              20640|            20640|             20640|             20640|            20640|             20640|            20640|             20640|             20640|              0|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488|537.8980135658915|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           null|
| stddev|  2.003531723502

In [52]:
def convert_categories(df):
    """
    Converts all string categories values into 0, 1, 2, 3.

    Args:
    df: The DataFrame that contains the string categories values.

    Returns:
    A new DataFrame with the converted string categories values.
    """

    categories = {
        "No": 0,
        "Yes": 1,
        "Month-to-month": 2,
        "One year": 3,
        "Two year": 4
    }

    # Convert the string categories values to their corresponding values using the dictionary.
    for column in df.columns:
        if column in categories:
            df = df.withColumn(column, F.when(df[column] == categories[column], 1).otherwise(0))

    return df

In [53]:
df_pyspark = convert_categories(df_pyspark)

In [56]:
# adding column in data frame
df_pyspark.withColumn("new_col", df_pyspark['median_house_value'] * 2).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity| new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|905200.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|717000.0|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|704200.0|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300

In [57]:
df_pyspark.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|           null|
|  -122.25|   37.85|              

In [58]:
# drop the column
df_pyspark.drop('new_col')

DataFrame[longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double, ocean_proximity: string]

In [59]:
df_pyspark.withColumnRenamed("new_col", "double_charges").show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|           null|
|  -122.25|   37.85|              

### Handling missing values
* dropping columns
* dropping rows
* variour parameters in dropping functions
* handling missing values by mean, median and mode

In [60]:
df_pyspark.na.drop().show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [61]:
# whethere if any col value is nul of any row
df_pyspark.na.drop(how='any').show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [62]:
# whethere if all col value is nul of any row
df_pyspark.na.drop(how='all').show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|           null|
|  -122.25|   37.85|              

In [63]:
# threshold
# whethere if threshold number of col value is nul of any row
no_of_col_null_to_check = 2
df_pyspark.na.drop(how='any', thresh=no_of_col_null_to_check).show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|           null|
|  -122.25|   37.85|              

In [64]:
# which col need to affect by subset
df_pyspark.na.drop(how='any', subset=['ocean_proximity'])

DataFrame[longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double, ocean_proximity: string]

In [65]:
# filling the missing values
df_pyspark.na.fill('null', ['ocean_proximity']).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|           null|
|  -122.25|   37.85|              

In [66]:
df_pyspark.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|           null|
|  -122.25|   37.85|              

In [71]:
null_count_ocean_proximity = df_pyspark.select(sum(col("ocean_proximity").isNull().cast("int"))).collect()[0][0]
print(null_count_ocean_proximity)

20640


In [77]:
imputer = Imputer(
    inputCols = ["ocean_proximity"],
    outputCols = ["{}_imputed".format(c) for c in ["ocean_proximity"]]
).setStrategy("mean")

In [78]:
df_pyspark.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|           null|
|  -122.25|   37.85|              

### Filter operation

In [81]:
df_pyspark.drop("ocean_proximity").show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|
|  -122.25|   37.85|              52.0|     1627.0|         280.0|     565.0|     259.0|       3.8462|          342200.0|
|  -122.25|   37.85|    

In [84]:
condition = "total_bedrooms > 4"
df_pyspark.filter(condition=condition).select(['total_bedrooms', 'median_house_value']).show()

+--------------+------------------+
|total_bedrooms|median_house_value|
+--------------+------------------+
|         129.0|          452600.0|
|        1106.0|          358500.0|
|         190.0|          352100.0|
|         235.0|          341300.0|
|         280.0|          342200.0|
|         213.0|          269700.0|
|         489.0|          299200.0|
|         687.0|          241400.0|
|         665.0|          226700.0|
|         707.0|          261100.0|
|         434.0|          281500.0|
|         752.0|          241800.0|
|         474.0|          213500.0|
|         191.0|          191300.0|
|         626.0|          159200.0|
|         283.0|          140000.0|
|         347.0|          152500.0|
|         293.0|          155500.0|
|         455.0|          158700.0|
|         298.0|          162900.0|
+--------------+------------------+
only showing top 20 rows



### Group by and aggregate

In [85]:
df_pyspark.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [86]:
# groupby
df_pyspark.groupBy("total_rooms", "median_house_value").sum().show()

+-----------+------------------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|total_rooms|median_house_value|sum(longitude)|sum(latitude)|sum(housing_median_age)|sum(total_rooms)|sum(total_bedrooms)|sum(population)|sum(households)|sum(median_income)|sum(median_house_value)|
+-----------+------------------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|     1238.0|           87500.0|       -122.28|        37.82|                   52.0|          1238.0|              288.0|          622.0|          259.0|            1.5057|                87500.0|
|     1848.0|          234000.0|       -122.26|        37.77|                   52.0|          1848.0|              479.0|          921.0|          477.0|             2.875|               234000.0|
|     1555

In [87]:
df_pyspark.groupBy("median_house_value").max().show()

+------------------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|median_house_value|max(longitude)|max(latitude)|max(housing_median_age)|max(total_rooms)|max(total_bedrooms)|max(population)|max(households)|max(median_income)|max(median_house_value)|
+------------------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|          270100.0|       -117.87|        37.81|                   52.0|          3789.0|              874.0|         2243.0|          866.0|            7.1472|               270100.0|
|          330000.0|       -117.69|        37.76|                   40.0|          6678.0|             1011.0|         2877.0|          982.0|            8.2787|               330000.0|
|          179300.0|       -117.92|        37.94|                   52

In [89]:
df_pyspark.groupBy("total_bedrooms").mean().show()

+--------------+-------------------+------------------+-----------------------+------------------+-------------------+------------------+------------------+------------------+-----------------------+
|total_bedrooms|     avg(longitude)|     avg(latitude)|avg(housing_median_age)|  avg(total_rooms)|avg(total_bedrooms)|   avg(population)|   avg(households)|avg(median_income)|avg(median_house_value)|
+--------------+-------------------+------------------+-----------------------+------------------+-------------------+------------------+------------------+------------------+-----------------------+
|         299.0|-119.98315789473686| 36.10026315789473|      31.07894736842105|1605.9473684210527|              299.0| 874.9736842105264|293.10526315789474| 4.308194736842106|     223418.55263157896|
|         305.0|-119.60423076923075| 35.36115384615385|      35.15384615384615|1469.1923076923076|              305.0| 887.6923076923077| 292.5769230769231| 3.761111538461539|      235569.3076923077|


In [90]:
df_pyspark.groupBy("total_rooms").count().show()

+-----------+-----+
|total_rooms|count|
+-----------+-----+
|      769.0|    4|
|     2862.0|    5|
|     3980.0|    2|
|     1761.0|    6|
|      692.0|    2|
|     4800.0|    1|
|      496.0|    1|
|      934.0|    5|
|     2734.0|    6|
|     1051.0|    8|
|      299.0|    3|
|     2815.0|    6|
|     4066.0|    2|
|     7554.0|    1|
|     5776.0|    1|
|      305.0|    1|
|     6433.0|    1|
|    12467.0|    1|
|     5983.0|    2|
|     4142.0|    1|
+-----------+-----+
only showing top 20 rows



In [92]:
df_pyspark.agg({'median_house_value':'sum'}).show()

+-----------------------+
|sum(median_house_value)|
+-----------------------+
|          4.269504061E9|
+-----------------------+



In [93]:
df_pyspark.groupBy("median_house_value").min().show()

+------------------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|median_house_value|min(longitude)|min(latitude)|min(housing_median_age)|min(total_rooms)|min(total_bedrooms)|min(population)|min(households)|min(median_income)|min(median_house_value)|
+------------------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|          270100.0|       -122.29|        33.78|                    9.0|          1142.0|              162.0|          486.0|          150.0|             4.072|               270100.0|
|          330000.0|       -122.47|        33.58|                    5.0|            67.0|               15.0|           49.0|           11.0|            3.1563|               330000.0|
|          179300.0|       -122.29|        33.83|                   21

In [95]:
df_pyspark.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|           null|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|           null|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|           null|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|           null|
|  -122.25|   37.85|              

In [153]:
# Combine independent columns into a single column with comma-separated values
df_combined = df_pyspark.withColumn("independent_features", concat_ws(",", "housing_median_age", "total_rooms", "total_bedrooms", "median_income"))

# Convert the combined values to Vector type
assembler = VectorAssembler(inputCols=["housing_median_age", "total_rooms", "total_bedrooms", "median_income"], outputCol="independent_features_vec")
df_combined = assembler.transform(df_combined)

# Drop unnecessary columns
df_combined = df_combined.drop("independent_features", "housing_median_age", "total_rooms", "total_bedrooms", "median_income")

# Rename the column
df_combined = df_combined.withColumnRenamed("independent_features_vec", "independent_features")

# Select specific columns
df_combined = df_combined.select("independent_features", "median_house_value")

# Show the modified DataFrame
df_combined.show(truncate=False)

+---------------------------+------------------+
|independent_features       |median_house_value|
+---------------------------+------------------+
|[41.0,880.0,129.0,8.3252]  |452600.0          |
|[21.0,7099.0,1106.0,8.3014]|358500.0          |
|[52.0,1467.0,190.0,7.2574] |352100.0          |
|[52.0,1274.0,235.0,5.6431] |341300.0          |
|[52.0,1627.0,280.0,3.8462] |342200.0          |
|[52.0,919.0,213.0,4.0368]  |269700.0          |
|[52.0,2535.0,489.0,3.6591] |299200.0          |
|[52.0,3104.0,687.0,3.12]   |241400.0          |
|[42.0,2555.0,665.0,2.0804] |226700.0          |
|[52.0,3549.0,707.0,3.6912] |261100.0          |
|[52.0,2202.0,434.0,3.2031] |281500.0          |
|[52.0,3503.0,752.0,3.2705] |241800.0          |
|[52.0,2491.0,474.0,3.075]  |213500.0          |
|[52.0,696.0,191.0,2.6736]  |191300.0          |
|[52.0,2643.0,626.0,1.9167] |159200.0          |
|[50.0,1120.0,283.0,2.125]  |140000.0          |
|[52.0,1966.0,347.0,2.775]  |152500.0          |
|[52.0,1228.0,293.0,

train_data, test_data = df_combined

In [154]:
train_data, test_data = df_combined.randomSplit([0.8, 0.2])

In [155]:
regressor = LinearRegression(featuresCol="independent_features", labelCol="median_house_value")

In [156]:
regressor = regressor.fit(train_data)

23/06/18 16:51:06 WARN Instrumentation: [ff29563f] regParam is zero, which might cause numerical instability and overfitting.
23/06/18 16:51:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/18 16:51:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/06/18 16:51:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [158]:
regressor.coefficients

DenseVector([1965.5078, -26.0187, 159.8721, 49198.7676])

In [159]:
regressor.intercept

-57838.98215750282

In [160]:
# prediction
pred_result = regressor.evaluate(test_data)

In [162]:
pred_result.predictions.show()

+--------------------+------------------+------------------+
|independent_features|median_house_value|        prediction|
+--------------------+------------------+------------------+
|[1.0,83.0,15.0,4....|          141700.0|184209.04509255738|
|[2.0,337.0,55.0,3...|          164800.0| 98839.50277607984|
|[2.0,718.0,98.0,4.1]|          315000.0| 144793.0008557141|
|[2.0,2239.0,321.0...|          240200.0|222047.96318348768|
|[2.0,2629.0,509.0...|          158500.0|167474.60389988238|
|[2.0,2682.0,393.0...|          196700.0|234007.43890135156|
|[2.0,4953.0,735.0...|          205100.0|182423.39173439218|
|[2.0,6718.0,858.0...|          305200.0|247310.24161602042|
|[2.0,7401.0,1187....|          177300.0| 146909.6458486813|
|[2.0,17978.0,3217...|          220800.0|246968.96396465725|
|[3.0,1629.0,317.0...|           83200.0| 159508.9651025134|
|[3.0,2516.0,429.0...|          271600.0|227661.87295465104|
|[3.0,4430.0,841.0...|          272700.0| 267157.7928043115|
|[3.0,5710.0,919.0...|  